In [1]:
from ctrace import *
import pandas as pd
import numpy as np

In [2]:
# Load in results file
with open(PROJECT_ROOT /"output"/"run_6Mm2w/" / "results.csv", "r") as csv:
    df = pd.read_csv(csv)
print(len(df))
df.head()

11200


,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap
0,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,88.819097,114.2,0.601694,661.962407,53
1,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,1217,6233,11135,16931,28,61.833362,85.2,0.921509,661.962407,405
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426
3,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,63.834657,81.0,0.322041,661.962407,-1
4,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,2359,25664,32967,145552,59,523.116239,1423.6,3.947826,3733.643370,15


In [3]:
# Extract the Gurobi MILP optimal runs
optimal = df[df["method"] == "mip_gurobi"]
print(len(optimal))
optimal.head()

2800


,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426
11,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1584,7527,12026,19278,27,74.677546,101.4,1.439489,846.461136,412
25,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,5772,26012,16285,42903,31,242.382803,465.8,7.878851,4839.647687,325
28,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,4854,31285,22040,82500,32,361.245810,875.4,14.611467,5702.689558,287
33,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,5736,26662,16890,47422,27,261.335158,515.2,12.540073,5014.496636,289


In [4]:
# Remove duplicates from different trials
optimal = optimal[optimal["trial_id"] == 0]

# Only these 4 ("G", "p", "budget", "from_cache") are needed to uniquely identify simulation parameters
optimal = optimal[["G", "p", "budget", "from_cache", "mip_value"]]
optimal = optimal.rename(columns={"mip_value": "optimal_mip_value"})

print(len(optimal))
optimal.head()

2800


,G,p,budget,from_cache,optimal_mip_value
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,60.874122
11,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,74.677546
25,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,242.382803
28,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,361.245810
33,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,261.335158


In [5]:
# Merge on the aforementioned key columns
df = df.merge(optimal, on=["G", "p", "budget", "from_cache"])

In [6]:
df.head()


,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value
0,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,88.819097,114.2,0.601694,661.962407,53,60.874122
1,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,1217,6233,11135,16931,28,61.833362,85.2,0.921509,661.962407,405,60.874122
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426,60.874122
3,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,63.834657,81.0,0.322041,661.962407,-1,60.874122
4,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,2359,25664,32967,145552,59,523.116239,1423.6,3.947826,3733.643370,15,432.438811


In [7]:
df["mip_ratio_gap"] = df["mip_value"] / df["optimal_mip_value"]
df.head()

,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap
0,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,1217,6233,11135,16931,28,88.819097,114.2,0.601694,661.962407,53,60.874122,1.459062
1,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,1217,6233,11135,16931,28,61.833362,85.2,0.921509,661.962407,405,60.874122,1.015758
2,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,1217,6233,11135,16931,28,60.874122,77.4,1.060092,661.962407,426,60.874122,1.000000
3,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,1217,6233,11135,16931,28,63.834657,81.0,0.322041,661.962407,-1,60.874122,1.048634
4,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,2359,25664,32967,145552,59,523.116239,1423.6,3.947826,3733.643370,15,432.438811,1.209688


In [8]:
# Check that mip_ratio_gap >= 1 for all elements
assert len(df[df["mip_ratio_gap"] < 1]) == 0

In [9]:
df = df.sort_values(by=['v2_size'])

In [10]:
dfk = [df[df["budget"] == x] for x in range(600, 1201, 100)]
dfk[0].head()

,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap
1356,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,40.197954,50.6,0.308147,278.972063,123,21.503628,1.869357
1357,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,21.619965,27.2,0.779478,278.972063,489,21.503628,1.005410
1358,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,greedy_weighted,0,538,2886,6427,8390,25,22.388281,33.0,0.017547,278.972063,-1,21.503628,1.041140
1359,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,538,2886,6427,8390,25,21.503628,26.2,0.734187,278.972063,500,21.503628,1.000000
758,montgomery,0.078,600,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,529,2808,6472,8273,23,21.620672,26.0,0.818339,276.134672,509,21.620672,1.000000


In [11]:
# Create an column that splits dataset into n groups
num_splits = 4
col = []
for i in range(num_splits):
    col += [i] * int(len(df) / num_splits)
df["v2_id"] = col

In [12]:
df.head()

,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap,v2_id
6159,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,14.974574,18.0,0.483384,278.972063,767,14.893481,1.005445,0
9357,montgomery,0.078,1100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,11.312337,13.8,0.431276,278.972063,978,11.271384,1.003633,0
9356,montgomery,0.078,1100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,32.454532,40.4,0.718345,278.972063,414,11.271384,2.879374,0
6158,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,538,2886,6427,8390,25,14.893481,16.8,0.402228,278.972063,792,14.893481,1.000000,0
6157,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,35.761233,45.6,0.313535,278.972063,285,14.893481,2.401133,0


In [95]:
dp = pd.pivot_table(df, values=["mip_ratio_gap", "min_exposed_value", "duration"], index=["method"], columns=["v2_id", "budget"], aggfunc=["mean", "max"])#%%

dp = dp.stack(0).stack(0)

In [96]:
dp = dp.reorder_levels([1, 0], 1)
dp = dp.sort_index(1)
dp.head()

budget                                  600                            \
v2_id                                      0           1            2   
method                                                                  
dependent max  duration             3.781748    8.078604    21.699885   
               min_exposed_value  238.400000  619.200000  1265.400000   
               mip_ratio_gap        1.038041    1.040865     1.051145   
          mean duration             1.830577    4.307887     9.824833   
               min_exposed_value  104.830000  382.268000   955.120000   

budget                                               700               \
v2_id                                       3           0           1   
method                                                                  
dependent max  duration             45.945911    5.258072    7.968174   
               min_exposed_value  1314.200000  230.200000  606.000000   
               mip_ratio_gap         1.077680    1.043510    1.045796   
          mean duration             17.348564    1.838484    4.362849   
               min_exposed_value  1017.812000  100.642000  376.536000   

budget                                                            800   \
v2_id                                       2            3           0   
method                                                                   
dependent max  duration             20.251029    50.380835    4.597445   
               min_exposed_value  1245.400000  1305.600000  229.600000   
               mip_ratio_gap         1.057797     1.085760    1.045829   
          mean duration             10.246147    18.092837    1.909075   
               min_exposed_value   942.026000   995.948000   95.670000   

budget                                        ...         1000               \
v2_id                                      1  ...            2            3   
method                                        ...                             
dependent max  duration             8.855141  ...    22.437100    60.053831   
               min_exposed_value  604.400000  ...  1210.600000  1255.600000   
               mip_ratio_gap        1.049818  ...     1.078036     1.112191   
          mean duration             4.649197  ...    11.783890    20.724335   
               min_exposed_value  367.842000  ...   907.964000   946.074000   

budget                                  1100                           \
v2_id                                      0           1            2   
method                                                                  
dependent max  duration             6.465027    9.106422    23.110142   
               min_exposed_value  207.200000  559.000000  1203.400000   
               mip_ratio_gap        1.058801    1.061818     1.078374   
          mean duration             2.155966    5.093305    12.346172   
               min_exposed_value   84.444000  347.998000   896.130000   

budget                                               1200              \
v2_id                                       3           0           1   
method                                                                  
dependent max  duration             57.725062    5.293113    9.956282   
               min_exposed_value  1225.200000  210.200000  559.000000   
               mip_ratio_gap         1.115501    1.061937    1.063669   
          mean duration             21.768375    2.183478    5.250280   
               min_exposed_value   931.472000   80.264000  340.568000   

budget                                                      
v2_id                                       2            3  
method                                                      
dependent max  duration             24.713740    57.852050  
               min_exposed_value  1192.400000  1219.000000  
               mip_ratio_gap         1.082586     1.131799  
          mean duration             12.976372    23.238314  
       

In [97]:
dp = dp.reorder_levels([0, 2, 1], 0)

In [98]:
dp = dp.sort_index(axis=0)

In [99]:
dp600 = dp[600]
dp600

v2_id                                            0           1            2  \
method                                                                        
dependent       duration          max     3.781748    8.078604    21.699885   
                                  mean    1.830577    4.307887     9.824833   
                min_exposed_value max   238.400000  619.200000  1265.400000   
                                  mean  104.830000  382.268000   955.120000   
                mip_ratio_gap     max     1.038041    1.040865     1.051145   
                                  mean    1.021244    1.032441     1.038486   
greedy_weighted duration          max     1.560015    1.139389     1.400481   
                                  mean    0.165830    0.398125     0.687424   
                min_exposed_value max   225.000000  599.400000  1210.800000   
                                  mean   99.064000  363.586000   906.338000   
                mip_ratio_gap     max     1.068697    1.054712     1.057458   
                                  mean    1.048383    1.046317     1.046540   
mip_gurobi      duration          max     6.737597   18.084417    45.786172   
                                  mean    2.394247    8.422776    20.841729   
                min_exposed_value max   243.000000  608.600000  1241.000000   
                                  mean  103.320000  376.374000   937.890000   
                mip_ratio_gap     max     1.000000    1.000000     1.000000   
                                  mean    1.000000    1.000000     1.000000   
random          duration          max     2.924506    5.877657     7.391596   
                                  mean    1.269207    2.924833     4.780821   
                min_exposed_value max   297.200000  698.800000  1401.200000   
                                  mean  139.016000  441.870000  1054.732000   
                mip_ratio_gap     max     1.869357    1.301130     1.204788   
                                  mean    1.467377    1.228974     1.174950   

v2_id                                             3  
method                                               
dependent       duration          max     45.945911  
                                  mean    17.348564  
                min_exposed_value max   1314.200000  
                                  mean  1017.812000  
                mip_ratio_gap     max      1.077680  
                                  mean     1.057720  
greedy_weighted duration          max      2.001919  
                                  mean     0.482665  
                min_exposed_value max   1238.000000  
                                  mean   965.442000  
                mip_ratio_gap     max      1.062014  
                                  mean     1.041912  
mip_gurobi      duration          max     62.091209  
                                  mean    33.921642  
                min_exposed_value max   1273.000000  
                                  mean   993.028000  
                mip_ratio_gap     max      1.000000  
                                  mean     1.000000  
random          duration          max     13.162880  
                                  mean     5.487362  
                min_exposed_value max   1507.000000  
                                  mean  1166.488000  
                mip_ratio_gap     max      1.454691  
                                  mean     1.289964

In [100]:
dp = dp.drop("mip_gurobi")
dp = dp.drop(("random", "duration"))

In [101]:
dp.index.names = ["method", "statistic", "aggregation"]

In [102]:
idx = pd.IndexSlice
dp.loc[idx[:, "min_exposed_value", :], :] = dp.loc[idx[:, "min_exposed_value", :], :].applymap(lambda x: '{:.1f}'.format(x))
dp.loc[idx[:, "mip_ratio_gap", :], :] = dp.loc[idx[:, "mip_ratio_gap", :], :].applymap(lambda x: '{:.3f}'.format(x))
dp.loc[idx[:, "duration", :], :] = dp.loc[idx[:, "duration", :], :].applymap(lambda x: '{:.1f}'.format(x))

In [103]:
dp

budget                                          600                          \
v2_id                                              0      1       2       3   
method          statistic         aggregation                                 
dependent       duration          max            3.8    8.1    21.7    45.9   
                                  mean           1.8    4.3     9.8    17.3   
                min_exposed_value max          238.4  619.2  1265.4  1314.2   
                                  mean         104.8  382.3   955.1  1017.8   
                mip_ratio_gap     max          1.038  1.041   1.051   1.078   
                                  mean         1.021  1.032   1.038   1.058   
greedy_weighted duration          max            1.6    1.1     1.4     2.0   
                                  mean           0.2    0.4     0.7     0.5   
                min_exposed_value max          225.0  599.4  1210.8  1238.0   
                                  mean          99.1  363.6   906.3   965.4   
                mip_ratio_gap     max          1.069  1.055   1.057   1.062   
                                  mean         1.048  1.046   1.047   1.042   
random          min_exposed_value max          297.2  698.8  1401.2  1507.0   
                                  mean         139.0  441.9  1054.7  1166.5   
                mip_ratio_gap     max          1.869  1.301   1.205   1.455   
                                  mean         1.467  1.229   1.175   1.290   

budget                                          700                          \
v2_id                                              0      1       2       3   
method          statistic         aggregation                                 
dependent       duration          max            5.3    8.0    20.3    50.4   
                                  mean           1.8    4.4    10.2    18.1   
                min_exposed_value max          230.2  606.0  1245.4  1305.6   
                                  mean         100.6  376.5   942.0   995.9   
                mip_ratio_gap     max          1.044  1.046   1.058   1.086   
                                  mean         1.023  1.037   1.043   1.066   
greedy_weighted duration          max            1.0    1.3     1.7     2.1   
                                  mean           0.1    0.5     0.7     0.5   
                min_exposed_value max          216.6  557.8  1216.6  1221.0   
                                  mean          94.5  353.5   888.5   936.7   
                mip_ratio_gap     max          1.077  1.063   1.063   1.078   
                                  mean         1.052  1.052   1.052   1.049   
random          min_exposed_value max          292.4  674.8  1406.0  1470.8   
                                  mean         136.3  440.2  1050.3  1158.3   
                mip_ratio_gap     max          2.055  1.346   1.228   1.498   
                                  mean         1.537  1.258   1.196   1.322   

budget                                          800          ...    1000  \
v2_id                                              0      1  ...       2   
method          statistic         aggregation                ...           
dependent       duration          max            4.6    8.9  ...    22.4   
                                  mean           1.9    4.6  ...    11.8   
                min_exposed_value max          229.6  604.4  ...  1210.6   
                                  mean          95.7  367.8  ...   908.0   
                mip_ratio_gap     max          1.046  1.050  ...   1.078   
                                  mean         1.024  1.041  ...   1.056   
greedy_weighted duration          max            1.1    1.5  ...     2.3   
                                  mean           0.2    0.4  ...     0.8   
                min_exposed_value max          211.2  537.6  ...  1126.2   
                                  mean          89.6  342.6  ...   835.6   
        

In [112]:
print(dp[600].to_latex())
print(dp[700].to_latex())
print(dp[800].to_latex())
print(dp[900].to_latex())
print(dp[1000].to_latex())
print(dp[1100].to_latex())
print(dp[1200].to_latex())

\begin{tabular}{lllllll}
\toprule
       &               & v2\_id &      0 &      1 &       2 &       3 \\
method & statistic & aggregation &        &        &         &         \\
\midrule
dependent & duration & max &    3.8 &    8.1 &    21.7 &    45.9 \\
       &               & mean &    1.8 &    4.3 &     9.8 &    17.3 \\
       & min\_exposed\_value & max &  238.4 &  619.2 &  1265.4 &  1314.2 \\
       &               & mean &  104.8 &  382.3 &   955.1 &  1017.8 \\
       & mip\_ratio\_gap & max &  1.038 &  1.041 &   1.051 &   1.078 \\
       &               & mean &  1.021 &  1.032 &   1.038 &   1.058 \\
greedy\_weighted & duration & max &    1.6 &    1.1 &     1.4 &     2.0 \\
       &               & mean &    0.2 &    0.4 &     0.7 &     0.5 \\
       & min\_exposed\_value & max &  225.0 &  599.4 &  1210.8 &  1238.0 \\
       &               & mean &   99.1 &  363.6 &   906.3 &   965.4 \\
       & mip\_ratio\_gap & max &  1.069 &  1.055 &   1.057 &   1.062 \\
       &        

In [107]:
df.head()

,G,p,budget,from_cache,method,trial_id,I_size,v1_size,v2_size,num_cross_edges,maxD,mip_value,min_exposed_value,duration,v1_objective,greedy_overlap,optimal_mip_value,mip_ratio_gap,v2_id
6159,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,14.974574,18.0,0.483384,278.972063,767,14.893481,1.005445,0
9357,montgomery,0.078,1100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,dependent,0,538,2886,6427,8390,25,11.312337,13.8,0.431276,278.972063,978,11.271384,1.003633,0
9356,montgomery,0.078,1100,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,32.454532,40.4,0.718345,278.972063,414,11.271384,2.879374,0
6158,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,mip_gurobi,0,538,2886,6427,8390,25,14.893481,16.8,0.402228,278.972063,792,14.893481,1.000000,0
6157,montgomery,0.078,900,/home/ubuntu/ContactTracing/data/SIR_Cache/tim...,random,0,538,2886,6427,8390,25,35.761233,45.6,0.313535,278.972063,285,14.893481,2.401133,0


In [127]:
si = df.pivot_table(values=["I_size", "v1_size", "v2_size", "num_cross_edges", "maxD"], columns="v2_id")
si.head()

v2_id,0,1,2,3
I_size,1552.37,4913.26,4458.85,1354.47
maxD,26.03,26.30,37.37,60.50
num_cross_edges,17897.95,38343.67,93190.25,134751.03
v1_size,7333.01,21556.44,30061.23,16893.83
v2_size,11016.06,15727.59,23848.67,37587.01


In [128]:
si.loc["I_size"] = si.loc["I_size"].apply(lambda x: '{:.1f}'.format(x / 1000))
si.loc["v1_size"] = si.loc["v1_size"].apply(lambda x: '{:.1f}'.format(x / 1000))
si.loc["v2_size"] = si.loc["v2_size"].apply(lambda x: '{:.1f}'.format(x / 1000))
si.loc["num_cross_edges"] = si.loc["num_cross_edges"].apply(lambda x: '{:.1f}'.format(x / 1000))
si.loc["maxD"] = si.loc["maxD"].apply(lambda x: '{:.1f}'.format(x))

In [129]:
print(si.to_latex())

\begin{tabular}{lllll}
\toprule
v2\_id &     0 &     1 &     2 &      3 \\
\midrule
I\_size          &   1.6 &   4.9 &   4.5 &    1.4 \\
maxD            &  26.0 &  26.3 &  37.4 &   60.5 \\
num\_cross\_edges &  17.9 &  38.3 &  93.2 &  134.8 \\
v1\_size         &   7.3 &  21.6 &  30.1 &   16.9 \\
v2\_size         &  11.0 &  15.7 &  23.8 &   37.6 \\
\bottomrule
\end{tabular}

